# 样例介绍
* 自动语音识别，即ASR，指借助计算机将语音转换为文本。在这个样例中，我们使用了基于深度学习的语音识别模型WeNet，借助我们的昇腾Atlas 200I DK A2，可以进行高性能推理。

# 前期准备
* 基础镜像的样例目录中已包含转换后的om模型以及测试文件，如果直接运行，可跳过此步骤。如果需要重新转换模型，可参考如下步骤：
* **建议在Linux服务器或者虚拟机转换该模型。**
* 首先我们可以在[这个链接](https://ascend-repo.obs.cn-east-2.myhuaweicloud.com/Atlas%20200I%20DK%20A2/DevKit/downloads/23.0.RC1/Ascend-devkit_23.0.RC1_downloads.xlsx)的表格中找到本样例的依赖文件，下载我们已经准备好了的ONNX模型文件，ONNX是开源的离线推理模型框架。

* 为了能进一步优化模型推理性能，我们需要将其转换为om模型进行使用，以下为转换指令：  
    ```shell
    atc --model=offline_encoder_sim.onnx --framework=5 --output=offline_encoder --input_format=ND --input_shape="speech:1,1478,80;speech_lengths:1" --log=error --soc_version=Ascend310B1
    ```
    其中转换参数的含义为：
    * --model：输入模型路径
    * --framework：原始网络模型框架类型，5表示ONNX
    * --output：输出模型路径
    * --input_format：输入Tensor的内存排列方式
    * --input_shape：指定模型输入数据的shape。这里我们在转模型的时候指定了最大的输入音频长度，推荐的长度有：262,326,390,454,518,582,646,710,774,838,902,966,1028,1284,1478，默认使用的长度是1478
    * --log：日志级别
    * --soc_version：昇腾AI处理器型号

# 模型推理实现

In [1]:
# 导入代码依赖
import torchaudio
import torchaudio.compliance.kaldi as kaldi
from ais_bench.infer.interface import InferSession
import numpy as np
import IPython

In [2]:
class WeNetASR:
    def __init__(self, model_path, vocab_path):
        """初始化模型，加载词表"""
        self.vocabulary = load_vocab(vocab_path)
        self.model = InferSession(0, model_path)
        # 获取模型输入特征的最大长度
        self.max_len = self.model.get_inputs()[0].shape[1]

    def transcribe(self, wav_file):
        """执行模型推理，将录音文件转为文本。"""
        feats_pad, feats_lengths = self.preprocess(wav_file)
        output = self.model.infer([feats_pad, feats_lengths])
        txt = self.post_process(output)
        return txt

    def preprocess(self, wav_file):
        """数据预处理"""
        waveform, sample_rate = torchaudio.load(wav_file)
        # 音频重采样，采样率16000
        waveform, sample_rate = resample(waveform, sample_rate, resample_rate=16000)
        # 计算fbank特征
        feature = compute_fbank(waveform, sample_rate)
        feats_lengths = np.array([feature.shape[0]]).astype(np.int32)
        # 对输入特征进行padding，使符合模型输入尺寸
        feats_pad = pad_sequence(feature,
                                 batch_first=True,
                                 padding_value=0,
                                 max_len=self.max_len)
        feats_pad = feats_pad.numpy().astype(np.float32)
        return feats_pad, feats_lengths

    def post_process(self, output):
        """对模型推理结果进行后处理，根据贪心策略选择概率最大的token，去除重复字符和空白字符，得到最终文本。"""
        encoder_out_lens, probs_idx = output[1], output[4]
        token_idx_list = probs_idx[0, :, 0][:encoder_out_lens[0]]
        token_idx_list = remove_duplicates_and_blank(token_idx_list)
        text = ''.join(self.vocabulary[token_idx_list])
        return text


def remove_duplicates_and_blank(token_idx_list):
    """去除重复字符和空白字符"""
    res = []
    cur = 0
    BLANK_ID = 0
    while cur < len(token_idx_list):
        if token_idx_list[cur] != BLANK_ID:
            res.append(token_idx_list[cur])
        prev = cur
        while cur < len(token_idx_list) and token_idx_list[cur] == token_idx_list[prev]:
            cur += 1
    return res


def pad_sequence(seq_feature, batch_first=True, padding_value=0, max_len=966):
    """对输入特征进行padding，使符合模型输入尺寸"""
    feature_shape = seq_feature.shape
    feat_len = feature_shape[0]
    if feat_len > max_len:
        # 如果输入特征长度大于模型输入尺寸，则截断
        seq_feature = seq_feature[:max_len].unsqueeze(0)
        return seq_feature

    batch_size = 1
    trailing_dims = feature_shape[1:]
    if batch_first:
        out_dims = (batch_size, max_len) + trailing_dims
    else:
        out_dims = (max_len, batch_size) + trailing_dims

    out_tensor = seq_feature.data.new(*out_dims).fill_(padding_value)
    if batch_first:
        out_tensor[0, :feat_len, ...] = seq_feature
    else:
        out_tensor[:feat_len, 0, ...] = seq_feature
    return out_tensor


def resample(waveform, sample_rate, resample_rate=16000):
    """音频重采样"""
    waveform = torchaudio.transforms.Resample(
        orig_freq=sample_rate, new_freq=resample_rate)(waveform)
    return waveform, resample_rate


def compute_fbank(waveform,
                  sample_rate,
                  num_mel_bins=80,
                  frame_length=25,
                  frame_shift=10,
                  dither=0.0):
    """提取filter bank音频特征"""
    AMPLIFY_FACTOR = 1 << 15
    waveform = waveform * AMPLIFY_FACTOR
    mat = kaldi.fbank(waveform,
                      num_mel_bins=num_mel_bins,
                      frame_length=frame_length,
                      frame_shift=frame_shift,
                      dither=dither,
                      energy_floor=0.0,
                      sample_frequency=sample_rate)
    return mat


def load_vocab(txt_path):
    """加载词表"""
    vocabulary = []
    LEN_OF_VALID_FORMAT = 2
    with open(txt_path, 'r') as fin:
        for line in fin:
            arr = line.strip().split()
            # 词表格式：token id
            if len(arr) != LEN_OF_VALID_FORMAT:
                raise ValueError(f"Invalid line: {line}. Expect format: token id")
            vocabulary.append(arr[0])
    return np.array(vocabulary)

# 样例运行

* 初始化相关参数

In [3]:
model_path = "offline_encoder.om"
vocab_path = 'vocab.txt'

model = WeNetASR(model_path, vocab_path)

* 展示样例语音

In [4]:
wav_file = 'sample.wav'

IPython.display.Audio(wav_file)

* 执行模型推理，打印识别文本。从推理结果大家可以看到识别出的文本跟上面的语音内容是一致的。

In [5]:
txt = model.transcribe(wav_file)
print(txt)

[INFO] acl init success
[INFO] open device 0 success
[INFO] load model offline_encoder.om success
[INFO] create model description success
智能语音作为智能时代人机交互的关键接口各行各业爆发式的场景需求驱动行业发展进入黄金期


2024-04-17 14:42:12.230198: E external/org_tensorflow/tensorflow/core/framework/node_def_util.cc:675] NodeDef mentions attribute input_para_type_list which is not in the op definition: Op<name=Sum; signature=input:T, reduction_indices:Tidx -> output:T; attr=keep_dims:bool,default=false; attr=T:type,allowed=[DT_FLOAT, DT_DOUBLE, DT_INT32, DT_UINT8, DT_INT16, 6034766930529145842, DT_UINT16, DT_COMPLEX128, DT_HALF, DT_UINT32, DT_UINT64]; attr=Tidx:type,default=DT_INT32,allowed=[DT_INT32, DT_INT64]> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node PartitionedCall_/ReduceSum_ReduceSum_670}}


# 样例总结与扩展
以上就是这个样例的全部内容了，值得注意的是，由于我们这里用的是静态shape的模型，即模型输入尺寸是固定的，所以在预处理的时候，如果输入的音频长度跟模型输入不同，需要通过padding或截断使其满足模型输入尺寸。大家可以尝试着为该语音识别模型添加下游任务，比如语音控制智能家居等。